In [ ]:
! pip install -q -U bitsandbytes
! pip install -q -U datasets
! pip install -q -U git+https://github.com/huggingface/transformers.git
! pip install -q -U git+https://github.com/huggingface/peft.git
! pip install -q -U git+https://github.com/huggingface/accelerate.git
! pip install -q -U loralib
! pip install -q -U einops

In [2]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from pprint import pprint
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
  LoraConfig,
  PeftConfig,
  PeftModel,
  get_peft_model,
  prepare_model_for_kbit_training
)
from transformers import (
  AutoConfig,
  AutoModelForCausalLM,
  AutoTokenizer,
  BitsAndBytesConfig
)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
MODEL_NAME = "vilm/vinallama-7b-chat"
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config = bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
config = PeftConfig.from_pretrained("kiendt/vinallama-math-7b")
model = PeftModel.from_pretrained(model, "kiendt/vinallama-math-7b")

adapter_config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

In [5]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [6]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

prompt = """
<| im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.
<|im_start|>user
### Câu hỏi:
Mua 15 cái bút hết 45 000 đồng. Vậy mua 40 cái bút hết số tiền là:
### Các lựa chọn:
A. 120 000 đồng
B. 240 000 đồng
C. 675 000 đồng
D. 15 000 đồng
### Câu trả lời:

<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors = "pt").to(device)
with torch.inference_mode() :
  outputs = model.generate(input_ids = encoding.input_ids,
                              attention_mask = encoding.attention_mask,
                              generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<| im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.
<|im_start|> user
### Câu hỏi:
Mua 15 cái bút hết 45 000 đồng. Vậy mua 40 cái bút hết số tiền là:
### Các lựa chọn:
A. 120 000 đồng
B. 240 000 đồng
C. 675 000 đồng
D. 15 000 đồng
### Câu trả lời:

<|im_start|> assistant
Đáp án A Mua 15 cái bút hết 45 000 đồng. Vậy mua 40 cái bút hết số tiền là: 45 000 – 15 000 = 120 000 đồng. Đáp số: 120 000 đồng. Đáp án A. Đáp án đúng là: 120 000 đồng. Vậy ta chọn đáp án: A. 120 000 đồng. Đáp án cần chọn là: A. Đáp án cần chọn là: A. 120 000 đồng. Đáp án cần chọn là: A. 120 000 đồng. Đáp án cần chọn là: A. 120 000 đồng. Đáp án cần chọn là: A. 120 000 đồng. Đáp án cần chọn là: A. 120 000 đồng. Đáp án cần chọn là: A. 120 000 đồng. Đáp án cần chọn là: A. 120 000 đồng. Đáp án cần chọn là: A. 120 000 đô


In [8]:
prompt = """
<| im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.
<|im_start|>user
### Câu hỏi:
Nhân viên y tế mất khoảng 5 phút để tiêm xong vắc-xin cho một người. Bố xếp hàng đợi tiêm lúc 9 giờ 20 phút. Phía trước bố còn 6 người nữ. Vậy bố sẽ được tiêm vắc-xin xong lúc:
### Các lựa chọn:
A. 9 giờ 50 phút
B. 10 giờ 5 phút
C. 10 giờ kém 5 phút
D. 9 giờ 25 phút
### Câu trả lời:

<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors = "pt").to(device)
with torch.inference_mode() :
  outputs = model.generate(input_ids = encoding.input_ids,
                              attention_mask = encoding.attention_mask,
                              generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

<| im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.
<|im_start|> user
### Câu hỏi:
Nhân viên y tế mất khoảng 5 phút để tiêm xong vắc-xin cho một người. Bố xếp hàng đợi tiêm lúc 9 giờ 20 phút. Phía trước bố còn 6 người nữ. Vậy bố sẽ được tiêm vắc-xin xong lúc:
### Các lựa chọn:
A. 9 giờ 50 phút
B. 10 giờ 5 phút
C. 10 giờ kém 5 phút
D. 9 giờ 25 phút
### Câu trả lời:

<|im_start|> assistant
Đáp án C Vậy bố sẽ được tiêm vắc-xin xong lúc: 9 giờ kém 5 phút. Đáp số: 9 giờ kém 5 phút. Đáp án C. 
 Số người cần tiêm xong là: 6 + 1 = 7 (người) 
 Vậy bố cần tiêm xong vò̉ 6 người trong 5 giờ (vì bố xếp hàng đợi tiêm lúc 9 giờ 20 mìn). 
 Vậy bố cần tiêm xong vò̉ 6 người trong 5 giờ là: 5 : 5 = 1 giờ (vì 5 giờ = 1 giờ). 
 Vậy bố sẽ được tiêm xong vò̉ 6 người trong 1 giờ 5 phút. 
 Vậy bố sẽ được tiêm xong vò̉ 6 người trong 5


In [18]:
prompt = """
<| im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.
<|im_start|>user
### Câu hỏi:
Ngày thứ nhất chú Tài thu mua được 1 350 kg phế liệu, ngày thứ hai thu mua được nhiều hơn ngày thứ nhất 580 kg phế liệu. Vậy sau hai ngày chú Tài thu mua được số ki-lô-gam phế liệu là:
### Các lựa chọn:
A. 1 930 kg
B. 3 280 kg
C. 1 830 kg
D. 2 280 kg
### Câu trả lời:

<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors = "pt").to(device)
with torch.inference_mode():
  outputs = model.generate(input_ids = encoding.input_ids,
                              attention_mask = encoding.attention_mask,
                              generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

<| im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.
<|im_start|> user
### Câu hỏi:
Ngày thứ nhất chú Tài thu mua được 1 350 kg phế liệu, ngày thứ hai thu mua được nhiều hơn ngày thứ nhất 580 kg phế liệu. Vậy sau hai ngày chú Tài thu mua được số ki-lô-gam phế liệu là:
### Các lựa chọn:
A. 1 930 kg
B. 3 280 kg
C. 1 830 kg
D. 2 280 kg
### Câu trả lời:

<|im_start|> assistant
Đáp án B Chú Tài thu mua được số ki-lô-gam phế liệu là: 1 350 + 580 = 2 280 (kg) Ngày thứ hai thu mua được số ki-lô-gam phế liệu là: 2 280 – 1 350 = 880 (kg) Ngày thứ ba thu mua được số ki-lô-gam phế liệu là: 880 + 580 = 3 280 (kg) Chú Tài thu mua được số ki-lô-gam phế liệu là: 1 350 + 580 + 880 = 2 280 (kg) Đáp số: 2 280 kg. Đáp án cần chọn là: B.  . Đáp án cần chọn là: B.  . Đáp án cần chọn là: B.  . Đáp án cần chọn là: B.  . Đáp án cần chọn là: B.  . Đáp án cần chọn là: B.  . Đáp án cầ